<a href="https://colab.research.google.com/github/dennisbonesso/DIO_sqlAlchemy/blob/main/sqlalchemy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
import inspect

from sqlalchemy.orm import declarative_base
from sqlalchemy.orm import Session
from sqlalchemy.orm import relationship
from sqlalchemy import Column
from sqlalchemy import Integer
from sqlalchemy import String
from sqlalchemy import ForeignKey
from sqlalchemy import create_engine
from sqlalchemy import inspect
from sqlalchemy.sql.expression import select



In [46]:
Base = declarative_base()

In [47]:
class User(Base):
      __tablename__ = "user_account"
      id = Column(Integer,primary_key=True)
      name = Column(String)
      fullname = Column(String)

      address = relationship(
          "Address", back_populates="user",cascade="all, delete-orphan"
      )

      def __repr__(self):
          return f"User(id={self.id},name={self.name},fullname={self.fullname})"



In [48]:
class Address(Base):
      __tablename__ = "address"
      id = Column(Integer,primary_key=True)
      email_address = Column(String(30),nullable=False)
      user_id=Column(Integer,ForeignKey("user_account.id"),nullable=False)

      user = relationship("User",back_populates="address")

      def __repr__(self):
          return f"Address(id={self.id},email={self.email_address})"

In [49]:
print(User.__tablename__)
print(Address.__tablename__)

user_account
address


In [50]:
engine = create_engine("sqlite://")

In [51]:
Base.metadata.create_all(engine)

In [52]:
insp_engine = inspect(engine)

In [53]:
print(insp_engine.get_columns("user_account")[0])

{'name': 'id', 'type': INTEGER(), 'nullable': False, 'default': None, 'autoincrement': 'auto', 'primary_key': 1}


In [54]:
print(insp_engine.default_schema_name)

main


In [55]:
with Session(engine) as session:
    dennis=User(
        name='dennis',
        fullname='dennis pires',
        address=[Address(email_address='dennis@email.com')]
    )

In [56]:
session.add_all([dennis])

In [57]:
session.commit()

In [59]:
stmt=select(User).where(User.name.in_(['dennis']))

In [62]:
for user in session.scalars(stmt):
  print(user)

User(id=1,name=dennis,fullname=dennis pires)


In [63]:
stmt_address=select(Address).where(Address.id.in_([1]))

In [64]:
for address in session.scalars(stmt_address):
  print(address)

Address(id=1,email=dennis@email.com)
